In [ ]:
import albumentations as alb
import copy
import time
import os
import cv2 
import numpy as np
from matplotlib import pyplot as plt
from math import log10, floor
import uuid

In [ ]:
original_images = os.path.join("main_folder","class3","images")
original_labels = os.path.join("main_folder","class3","labels")

In [ ]:
# Change this variable according to your output images you need
output_images = 13

In [1]:
# Set these paths if you want your own paths 
augmented_images = os.path.join(os.path.dirname(os.path.dirname(original_images)),"augmented_data","images")
augmented_labels = os.path.join(os.path.dirname(augmented_images),"labels")

print("Augmented dataset path : " + os.path.dirname(augmented_images))

NameError: name 'original_images' is not defined

In [ ]:
augmentor = alb.Compose([alb.RandomCrop(width=1280, height=720),
                         alb.HorizontalFlip(p=0.5),
                         alb.RandomBrightnessContrast(p=0.2),
                         alb.RandomGamma(p=0.2), 
                         alb.RGBShift(p=0.2), 
                         alb.VerticalFlip(p=0.5)], 
                       bbox_params=alb.BboxParams(format='yolo', 
                                                  label_fields=['class_labels'],
                                                  min_area=1024, #remove this if your objects are small
                                                  min_visibility=0.3)) # remove this contraint if your objects are small

In [ ]:
# function for writting every parameter in 8 digits according to YOLO default length
def round_off(n):
    n = str(round(n, 6-int(floor(log10(abs(n))))-1))
    if len(n) > 8:
        n = n[:8]
    elif len(n) == 3:
        n = n +"00000"
    elif len(n) == 4:
        n = n +"0000"
    elif len(n) == 5:
        n = n +"000"
    elif len(n) == 6:
        n = n +"00"
    elif len(n) == 7:
        n = n +"0"
    return n

In [ ]:
os.makedirs(augmented_images)
os.makedirs(augmented_labels)

for image in os.listdir(original_images): 
    
    orignal_image = cv2.imread(os.path.join(original_images,image)) 
    bbox_file=os.path.join(original_labels, image.split(".")[0]+".txt") 
    
    # comment it if output image color is not natural (optional)
    orignal_image = cv2.cvtColor(orignal_image, cv2.COLOR_BGR2RGB)
    
    # declarations
    org_bbox=[]
    class_labels =[]
    
    # getting bounding box from file into memory
    if os.path.exists(bbox_file):
        bbox_file =open(bbox_file,"r")
        for line in bbox_file:
            org_bbox.append(list(map(float,line.split())))
        bbox_file.close()
    else:
        continue
    
    # separating labels and bounding box in two different lists    
    for col in org_bbox:
        class_labels.append(col[0])
        del col[0]
        
    
    # This is the main loop
    try:
        for x in range(output_images):
            
            # main augmenting function
            augmented = augmentor(image=orignal_image, bboxes=org_bbox, class_labels=class_labels)
            
            # for different file name in every iteration but same for both image and txt file
            name = str(uuid.uuid4().hex)
            
            # checking if there is no bounding box then don't save that image and txt file
            if len(augmented['bboxes']) != 0:
                
                # for saving augmented image
                cv2.imwrite(os.path.join(augmented_images,f'{name}.jpg'), augmented['image'])
                
                # making txt file with same name as img file with .txt extention
                txt_file = open(os.path.join(augmented_labels,f'{name}.txt'),"w")
                
                # for saving parameters in txt file
                for items in augmented['bboxes']:
                    
                    # counter for each line in bbox
                    label = 0                
                    
                    # first writting label
                    txt_file.write(str(int(augmented['class_labels'][label])))
                    
                    # a space after label
                    txt_file.write(" ")
                    
                    # writing four parameters of bbox
                    for n in items:
                        
                        # writting every parameter in 8 digits according to YOLO default length
                        txt_file.write(round_off(n))
                        
                        # a space after parameter
                        txt_file.write(" ")
                        
                    # writting untill all parameters against each class are not written
                    if label < len(augmented['bboxes']) :
                        txt_file.write("\n")
                        
                    # increment in counter
                    label += 1
                
                # clossing the txt file of bbox
                txt_file.close()
            
    # for dealing with any unexpected exception
    except Exception as e:
        print(e)
            